<a href="https://colab.research.google.com/github/AllenEdgarPoe/Computer_vision/blob/master/GRU_directional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### Libraries

In [0]:
!pip3 install torch unidecode
import torch
import torch.nn as nn
from torch.autograd import Variable
import unidecode
import string
import random
import re
import time
import math

     |████████████████████████████████| 245kB 3.4MB/s 


In [0]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 2
lr = 0.002

all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### Get Text

In [0]:
file = unidecode.unidecode(open("./gdrive/My Drive/data/linux.txt").read())
file_len = len(file)
print("file의 길이는: ", file_len)

file의 길이는:  34167


###Functions for text processing

 1. Random Chunk: 랜덤하게 200개 charcter 추출하기

In [0]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

holding bfqd->lock. A blkg_lookup performed with
	 * bfqd->lock held then returns a fully consistent blkg, which
	 * remains consistent until this lock is held.
	 *
	 * Thanks to the last fact, and to 


2. Character to tensor: char를 숫자 텐서로 표현하기.. 앞에서 100개 all printable 한 character를 구했으니까 0~100 사이 숫자로 표현될 것. index

In [0]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()       #먼저 100개의 0으로 구성된 텐서로 만들자
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return Variable(tensor).cuda()     #tensor로 만들어주기 

3. Chunk into input & label: 랜덤하게 구한 chunk를 input과 label로 나누기. training dataset을 만들기 위해..

In [0]:
def random_training_set(): 
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

### Model&Optimizer

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, hidden_size)).cuda()
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2).cuda()

In [43]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

tensor([36], device='cuda:0')
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### Test 

In [0]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

### Training

In [46]:
loss_list = []
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].view(-1)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor(4.5972, device='cuda:0', grad_fn=<DivBackward0>) 

bjc`N<b>r|A`-{dKRii9UQcKZY}
7n?TQobm<rqH@j}hA>_8 3m?o6V)-+c)LQY)zw_`#kv'ae{vzskhN&)bsS&s+v~65{tI
E(ieE/%&zC~bf}8+m']kGidv{wkDN;k]mc$o\"p6if
W27W}MhJaut}q#u~w}id'K|wiR5%y\p4vv-=mds;VyYFr'=rlC,FkI]}`5}



 tensor(1.6960, device='cuda:0', grad_fn=<DivBackward0>) 

bgipd_enled;
	ests_he   lom unlck t bfq_rents ast stoe);
>een;
 *bfq_Krtialent bfq_&ict *->ill = bfqg_pctoup;
		bfq_bfqg_storu->eiataenthI hc *statattag)

thny_trie_bqgs,
	= bfqq_pinty_tatiseon d  it(s



 tensor(1.9091, device='cuda:0', grad_fn=<DivBackward0>) 

bfqg) {
	rooup *blkg_que) {
	blkg_prbin_cololic the bfqd,
		 * pro_crlass = woo_pd_queseue(st
	.licy the the tles
	lcget_winight mand (int_the, Tfreouns pliolw(bfqq_pq_blkcg_proup *blkg_rows_uflig_wrou



 tensor(1.2422, device='cuda:0', grad_fn=<DivBackward0>) 

bRfqq);
			return blkcg, fold bfqg_stat_blkcg_pd_wait_tivate. it serivericy_sure the all = 0;
}

	blkg_roup *stats_blkcg_pit_stat, &stats->time);
	b